# Basic Loading of the Kernel

The Semantic Kernel SDK can be imported from the following nuget feed:

In [22]:
#r "nuget: Microsoft.SemanticKernel, 1.45.0"
#r "nuget: Microsoft.SemanticKernel.PromptTemplates.Handlebars, 1.45.0"
#r "nuget: Microsoft.SemanticKernel.Yaml, 1.45.0"

Installed Packages Microsoft.SemanticKernel, 1.45.0 Microsoft.SemanticKernel.PromptTemplates.Handlebars, 1.45.0 Microsoft.SemanticKernel.Yaml, 1.45.0

##### Instantiate Kernel, and fire away prompts


In [ ]:
using Microsoft.SemanticKernel;
using Microsoft.Extensions.Logging;
using Microsoft.Extensions.Logging.Abstractions;
using Microsoft.Extensions.DependencyInjection;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using Kernel = Microsoft.SemanticKernel.Kernel;
using OpenAI.Chat;

#!import config/Settings.cs 

var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();

var builder = Kernel.CreateBuilder();
builder.AddOpenAIChatCompletion(model, apiKey, orgId);
var kernel = builder.Build();

Console.WriteLine ( await kernel.InvokePromptAsync("Tell me a joke about cats") );

##### You can control the number of tokens you want to spend

In [ ]:
OpenAIPromptExecutionSettings settings = new OpenAIPromptExecutionSettings()
{
    Temperature = 0.9f,
    MaxTokens = 50,
    TopP = 0.9f,
};

var result = await kernel.InvokePromptAsync("Tell me a joke about cats", new (settings));
Console.WriteLine ( result );
Console.WriteLine ("TOKENS USED: "+ ((ChatTokenUsage) result.Metadata["Usage"]).OutputTokenCount );

##### Now, let us try templatized prompts.

In [ ]:
string promptTemplate = "Write funny tweet about {{$topic}}";

Console.WriteLine ( await kernel.InvokePromptAsync(promptTemplate, new() { { "topic", "people who run for fun" } } ) );
Console.WriteLine ( await kernel.InvokePromptAsync(promptTemplate, new() { { "topic", "seattle weather" } } ) );
Console.WriteLine ( await kernel.InvokePromptAsync(promptTemplate, new() { { "topic", "spicy noodels" } } ) );


##### You can bring in project scoped 'PromptFilters' to enforce policies

In [7]:
public class ResponsibleAIPromptFilter() : IPromptRenderFilter
    {        
        /// <summary>
        /// Method which is called asynchronously before prompt rendering.
        /// </summary>
        /// <param name="context">Instance of <see cref="PromptRenderContext"/> with prompt rendering details.</param>
        /// <param name="next">Delegate to the next filter in pipeline or prompt rendering operation itself. If it's not invoked, next filter or prompt rendering won't be invoked.</param>
        public async Task OnPromptRenderAsync(PromptRenderContext context, Func<PromptRenderContext, Task> next)
        {
            if (context.Arguments.ContainsName("card_number"))
            {
                context.Arguments["card_number"] = "**** **** **** ****";
            }

            await next(context);

            context.RenderedPrompt += " NO SEXISM, RACISM OR OTHER BIAS/BIGOTRY."; // Add this line to filter out bias or bigotry
            context.RenderedPrompt += " NO HATE SPEECH OR HARASSMENT."; // Add this line to filter out hate speech or harassment
            context.RenderedPrompt += " NO VIOLENCE OR HARMFUL CONTENT."; // Add this line to filter out violence or harmful content
            context.RenderedPrompt += " NO ILLEGAL OR UNETHICAL CONTENT."; // Add this line to filter out illegal or unethical content            
        }
    }

##### And now, some yaml!

In [ ]:
using Microsoft.SemanticKernel;
using Kernel = Microsoft.SemanticKernel.Kernel;
using Microsoft.SemanticKernel.PromptTemplates.Handlebars;
#!import config/Settings.cs 


var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();
var builder = Kernel.CreateBuilder();
builder.AddOpenAIChatCompletion(model, apiKey, orgId);
builder.Services.AddSingleton<IPromptRenderFilter, ResponsibleAIPromptFilter>();
var kernel = builder.Build();
        
// Load prompt from resources
StreamReader reader = new("resources/prompts/GenerateExcuse.yaml", System.Text.Encoding.UTF8);
string generateExcuseYaml = reader.ReadToEnd();
reader.Close();

KernelFunction function = kernel.CreateFunctionFromPromptYaml(generateExcuseYaml);

// Invoke the prompt function and display the result
Console.WriteLine(await kernel.InvokeAsync(function, arguments: new()
{
    { "event", "I did not submit connect" },
    { "length", "1" },
}));

       